In [16]:
import os
import json
import numpy as np
import pandas as pd
import umap
import hdbscan
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_mutual_info_score, silhouette_score, adjusted_rand_score, \
    normalized_mutual_info_score
from collections import Counter

In [17]:
def read_csv_files_to_dict(base_path, dimension, file_prefix, missing_percentages, strategies=None):
    dataframes_dict = {}

    for percentage in missing_percentages:
        percentage_key = f"{int(percentage * 100)}%"

        if dimension == 'Completeness':
            dataframes_dict[percentage_key] = {}

            if strategies is not None:
                for strategy_name, strategy_func, subfolder in strategies:
                    file_path = f"{base_path}/{dimension}/{subfolder}/{file_prefix}_{percentage_key}.csv"

                    try:
                        df = pd.read_csv(file_path)
                        dataframes_dict[percentage_key][strategy_name] = df
                    except FileNotFoundError:
                        print(f"File not found: {file_path}")
                        dataframes_dict[percentage_key][strategy_name] = None
            else:
                print("No strategies provided for Completeness dimension.")
        elif dimension == 'Unicity':
            file_path = f"{base_path}/{dimension}/{file_prefix}_{percentage_key}_2x.csv"
            try:
                df = pd.read_csv(file_path)
                dataframes_dict[percentage_key] = df
            except FileNotFoundError:
                print(f"File not found: {file_path}")
                dataframes_dict[percentage_key] = None
        else:
            file_path = f"{base_path}/{dimension}/{file_prefix}_{percentage_key}.csv"
            try:
                df = pd.read_csv(file_path)
                dataframes_dict[percentage_key] = df
            except FileNotFoundError:
                print(f"File not found: {file_path}")
                dataframes_dict[percentage_key] = None

    return dataframes_dict

def convert_to_float(obj):
    """
    Convertit les types de données non JSON-serializables en types natifs Python.
    """
    if isinstance(obj, np.float32) or isinstance(obj, np.float64):
        return float(obj)
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    return obj

def update_json_results(output_path, model_name, metrics, pollution_percentage_levels):
    # Charger le fichier JSON existant, ou initialiser une nouvelle structure si le fichier n'existe pas
    if os.path.exists(output_path):
        with open(output_path, 'r') as json_file:
            results_dict = json.load(json_file)
    else:
        results_dict = {
            "models": []
        }

    # Trouver ou ajouter l'entrée pour le modèle spécifié
    model_entry = next((model for model in results_dict["models"] if model["model"] == model_name), None)

    if not model_entry:
        model_entry = {
            "model": model_name,
            "pollution_metrics": []
        }
        results_dict["models"].append(model_entry)

    # Mise à jour pour la pollution à 0 (df_clean)
    pollution_percentage = 0
    existing_entry = next((item for item in model_entry["pollution_metrics"] if item["pollution_percentage"] == pollution_percentage), None)

    if existing_entry:
        # Remplacer les métriques
        existing_entry["metrics"] = {
            # "silhouette score": convert_to_float(metrics["Silhouette_Score_Clean"]),
            "stability indexes": 1.0,
            "ARI score": 1.0,
            "Cluster_Overlap_Size": 1.0,
            "Cluster_Size_Variance": 1.0
        }
    else:
        # Ajouter une nouvelle entrée pour ce pourcentage
        model_entry["pollution_metrics"].append({
            "pollution_percentage": pollution_percentage,
            "metrics": {
                # "silhouette score": convert_to_float(metrics["Silhouette_Score_Clean"]),
                "stability indexes": 1.0,
                "ARI score": 1.0,
                "Cluster_Overlap_Size": 1.0,
                "Cluster_Size_Variance": 1.0
            }
        })

    # Mise à jour pour les autres niveaux de pollution
    for i, pollution_percentage in enumerate(pollution_percentage_levels):
        pollution_percentage = int(pollution_percentage*100)
        existing_entry = next((item for item in model_entry["pollution_metrics"] if item["pollution_percentage"] == pollution_percentage), None)

        if existing_entry:
            # Remplacer les métriques
            existing_entry["metrics"] = {
                # "silhouette score": convert_to_float(metrics["Silhouette_Score_Noisy"][i]),
                "stability indexes": convert_to_float(metrics["Stability_Index"][i]),
                "ARI score": convert_to_float(metrics["ARI"][i]),
                "Cluster_Overlap_Size": convert_to_float(metrics["Cluster_Size_Variance"][i]),
                "Cluster_Size_Variance": convert_to_float(metrics["Cluster_Size_Variance"][i])
                
            }
        else:
            # Ajouter une nouvelle entrée pour ce pourcentage
            model_entry["pollution_metrics"].append({
                "pollution_percentage": pollution_percentage,
                "metrics": {
                    # "silhouette score": convert_to_float(metrics["Silhouette_Score_Noisy"][i]),
                    "stability indexes": convert_to_float(metrics["Stability_Index"][i]),
                    "ARI score": convert_to_float(metrics["ARI"][i]),
                    "Cluster_Overlap_Size": convert_to_float(metrics["Cluster_Size_Variance"][i]),
                    "Cluster_Size_Variance": convert_to_float(metrics["Cluster_Size_Variance"][i])
                }
            })

    # Écrire les résultats mis à jour dans le fichier JSON
    with open(output_path, 'w') as json_file:
        json.dump(results_dict, json_file, indent=4)

    print(f"Results saved to {output_path}")

def prepare_retail_data(df):
    # Séparation des caractéristiques numériques et catégorielles
    numeric_features = df.select_dtypes(include='number')
    categorical_features = df.select_dtypes(include='object')

    # Transformation des données numériques
    scaler = StandardScaler()
    numeric_data_scaled = scaler.fit_transform(numeric_features)

    # Transformation des données catégorielles en utilisant pd.get_dummies
    categorical_data_encoded = pd.get_dummies(categorical_features, drop_first=True)

    # Combinaison des données numériques et catégorielles transformées
    df_preprocessed = pd.concat([pd.DataFrame(numeric_data_scaled, columns=numeric_features.columns).reset_index(drop=True),
                                 categorical_data_encoded.reset_index(drop=True)], axis=1)

    return df_preprocessed

def compute_metrics(df_clean, df_noisy_dict, clustering_algo=hdbscan.HDBSCAN(min_cluster_size=10, min_samples=5)):
    metrics_results = {
        "Stability_Index": [],
        "ARI": [],
        "Cluster_Overlap_Size": [],
        "Cluster_Size_Variance": []
    }

    # 1. Réduction de dimensionnalité sur les données propres avec un nouveau modèle UMAP
    print("Réduction de dimensionnalité sur les données propres")
    umap_model_clean = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2)
    embedding_clean = umap_model_clean.fit_transform(df_clean)

    # 2. Clustering sur les données propres
    print("Clustering sur les données propres")
    cluster_labels_clean = clustering_algo.fit_predict(embedding_clean)

    for key in df_noisy_dict.keys():
        df_noisy = df_noisy_dict[key]

        if df_noisy is not None:
            # Downsample the noisy dataset to match the clean dataset row count
            if len(df_noisy) > len(df_clean):
                df_noisy_sampled = df_noisy.sample(n=len(df_clean), random_state=42)
            else:
                df_noisy_sampled = df_noisy

            # 3. Réduction de dimensionnalité sur les données bruitées
            print(f"Réduction de dimensionnalité sur les données bruitées: {key}")
            umap_model_noisy = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2)
            embedding_noisy = umap_model_clean.fit_transform(df_noisy_sampled)

            # 4. Clustering sur les données bruitées
            print(f"Clustering sur les données bruitées: {key}")
            cluster_labels_noisy = clustering_algo.fit_predict(embedding_noisy)

            # 5. Calcul des métriques

            # Adjusted Rand Index (ARI)
            print("Calcul du Adjusted Rand Index (ARI) sur les données bruitées")
            ari = adjusted_rand_score(cluster_labels_clean, cluster_labels_noisy)
            metrics_results["ARI"].append(ari)

            # Stability Index (SI)
            print("Calcul du Stability Index (SI) sur les données bruitées")
            stability_index = normalized_mutual_info_score(cluster_labels_clean, cluster_labels_noisy)
            metrics_results["Stability_Index"].append(stability_index)

            # Cluster Overlap Size
            print("Calcul de la taille absolue de l'overlap des clusters")
            overlap_size = compute_cluster_overlap(cluster_labels_clean, cluster_labels_noisy)
            metrics_results["Cluster_Overlap_Size"].append(overlap_size)

            # Cluster Size Variance
            print("Calcul de la variance de la taille des clusters")
            size_variance = compute_cluster_size_variance(cluster_labels_noisy)
            metrics_results["Cluster_Size_Variance"].append(size_variance)
        else:
            metrics_results["ARI"].append(None)
            metrics_results["Stability_Index"].append(None)
            metrics_results["Cluster_Overlap_Size"].append(None)
            metrics_results["Cluster_Size_Variance"].append(None)

    return metrics_results


def compute_cluster_overlap(labels1, labels2):
    overlap = 0
    for cluster1 in np.unique(labels1):
        for cluster2 in np.unique(labels2):
            intersection = np.sum((labels1 == cluster1) & (labels2 == cluster2))
            union = np.sum(labels1 == cluster1) + np.sum(labels2 == cluster2) - intersection
            if union > 0:
                overlap += intersection / union
    return overlap

def compute_cluster_size_variance(labels):
    sizes = [np.sum(labels == label) for label in np.unique(labels)]
    return np.var(sizes) if len(sizes) > 1 else 0


## Completeness

In [4]:
# Utilisation de la fonction dans votre code existant
missing_percentages = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

# Strategy :
strategy_delete = {
    'numerical': 'delete',
    'categorical': 'delete'
}

strategy_mean_mode = {
    'numerical': 'mean',
    'categorical': 'mode'
}

strategy_median_new = {
    'numerical': 'median',
    'categorical': 'new'
}

strategy_decision_tree = {
    'numerical': 'decision_tree',
    'categorical': 'decision_tree'
}

strategy_mean_new = {
    'numerical': 'mean',
    'categorical': 'new'
}

strategy_knn_mode = {
    'numerical': 'knn',
    'categorical': 'mode'
}

strategies = [
    ("Mean and Mode", strategy_mean_mode, "Mean and Mode"),
    ("Median and New", strategy_median_new, "Median and New"),
    ("Decision Tree", strategy_decision_tree, "Decision Tree"),
    ("Mean and New", strategy_mean_new, "Mean and New")
]

# Chargement du DataSet clean
retail_df_clean = pd.read_csv('../../Data/Clustering/retail Data/retail_data_clean.csv')
retail_df_clean = prepare_retail_data(retail_df_clean)

# Chargement du DataSet bruité
retail_df_noisy_dict = read_csv_files_to_dict('../../Data/Clustering/retail Data', 'Completeness', 'retail', missing_percentages, strategies)

for key, retail_df_strategies in retail_df_noisy_dict.items():
    for strategies , retail_df in retail_df_strategies.items():
        retail_df_noisy_dict[key][strategies] = prepare_retail_data(retail_df_noisy_dict[key][strategies])
        

# Chemin du fichier JSON où les résultats seront enregistrés
output_path = "../../Results/Retail/Completeness.json"

# # Calcule des metriques
for key, retail_df_strategies in retail_df_noisy_dict.items():
    print(key)
    metrics = compute_metrics(retail_df_clean, retail_df_noisy_dict[key])
    print(metrics)

20%
Réduction de dimensionnalité sur les données propres


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Clustering sur les données propres
Réduction de dimensionnalité sur les données bruitées: Mean and Mode
Clustering sur les données bruitées: Mean and Mode
Calcul du Adjusted Rand Index (ARI) sur les données bruitées
Calcul du Stability Index (SI) sur les données bruitées
Réduction de dimensionnalité sur les données bruitées: Median and New
Clustering sur les données bruitées: Median and New
Calcul du Adjusted Rand Index (ARI) sur les données bruitées
Calcul du Stability Index (SI) sur les données bruitées
Réduction de dimensionnalité sur les données bruitées: Decision Tree
Clustering sur les données bruitées: Decision Tree
Calcul du Adjusted Rand Index (ARI) sur les données bruitées
Calcul du Stability Index (SI) sur les données bruitées
Réduction de dimensionnalité sur les données bruitées: Mean and New
Clustering sur les données bruitées: Mean and New
Calcul du Adjusted Rand Index (ARI) sur les données bruitées
Calcul du Stability Index (SI) sur les données bruitées
{'Stability_Index

## Consistent Representation

In [18]:
output_path = "../../Results/Retail/Consistent Representation.json"
model_name = "HDBSCAN"

# Chargement du DataSet clean
retail_df_clean = pd.read_csv('../../Data/Clustering/retail Data/retail_data_clean.csv')
retail_df_clean = prepare_retail_data(retail_df_clean)

# Chargement des DataSet pollués
pollution_percentage_levels = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
retail_df_noisy_dict = read_csv_files_to_dict('../../Data/Clustering/retail Data', 'Consistent Representation', 'retail', pollution_percentage_levels)

for key in retail_df_noisy_dict.keys():
    if retail_df_noisy_dict[key] is not None:
        retail_df_noisy_dict[key] = prepare_retail_data(retail_df_noisy_dict[key])

metrics = compute_metrics(retail_df_clean, retail_df_noisy_dict)

update_json_results(output_path, model_name, metrics, pollution_percentage_levels)

Réduction de dimensionnalité sur les données propres
Clustering sur les données propres
Réduction de dimensionnalité sur les données bruitées: 10%
Clustering sur les données bruitées: 10%
Calcul du Adjusted Rand Index (ARI) sur les données bruitées
Calcul du Stability Index (SI) sur les données bruitées
Calcul de la taille absolue de l'overlap des clusters
Calcul de la variance de la taille des clusters
Réduction de dimensionnalité sur les données bruitées: 20%
Clustering sur les données bruitées: 20%
Calcul du Adjusted Rand Index (ARI) sur les données bruitées
Calcul du Stability Index (SI) sur les données bruitées
Calcul de la taille absolue de l'overlap des clusters
Calcul de la variance de la taille des clusters
Réduction de dimensionnalité sur les données bruitées: 30%
Clustering sur les données bruitées: 30%
Calcul du Adjusted Rand Index (ARI) sur les données bruitées
Calcul du Stability Index (SI) sur les données bruitées
Calcul de la taille absolue de l'overlap des clusters
Cal

## Feature Accuracy

In [15]:
output_path = "../../Results/Retail/Feature Accuracy.json"
model_name = "HDBSCAN"

# Chargement du DataSet clean
retail_df_clean = pd.read_csv('../../Data/Clustering/retail Data/retail_data_clean.csv')
retail_df_clean = prepare_retail_data(retail_df_clean)

# Chargement des DataSet pollués
pollution_percentage_levels = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
retail_df_noisy_dict = read_csv_files_to_dict('../../Data/Clustering/retail Data', 'Feature Accuracy', 'retail', pollution_percentage_levels)

for key in retail_df_noisy_dict.keys():
    if retail_df_noisy_dict[key] is not None:
        retail_df_noisy_dict[key] = prepare_retail_data(retail_df_noisy_dict[key])

metrics = compute_metrics(retail_df_clean, retail_df_noisy_dict)

update_json_results(output_path, model_name, metrics, pollution_percentage_levels)

Réduction de dimensionnalité sur les données propres
Clustering sur les données propres
Réduction de dimensionnalité sur les données bruitées: 10%
Clustering sur les données bruitées: 10%
Calcul du Adjusted Rand Index (ARI) sur les données bruitées
Calcul du Stability Index (SI) sur les données bruitées
Calcul de la taille absolue de l'overlap des clusters
Calcul de la variance de la taille des clusters
Réduction de dimensionnalité sur les données bruitées: 20%
Clustering sur les données bruitées: 20%
Calcul du Adjusted Rand Index (ARI) sur les données bruitées
Calcul du Stability Index (SI) sur les données bruitées
Calcul de la taille absolue de l'overlap des clusters
Calcul de la variance de la taille des clusters
Réduction de dimensionnalité sur les données bruitées: 30%
Clustering sur les données bruitées: 30%
Calcul du Adjusted Rand Index (ARI) sur les données bruitées
Calcul du Stability Index (SI) sur les données bruitées
Calcul de la taille absolue de l'overlap des clusters
Cal

## Unicity

In [19]:
output_path = "../../Results/Retail/Unicity.json"
model_name = "HDBSCAN"

# Chargement du DataSet clean
retail_df_clean = pd.read_csv('../../Data/Clustering/retail Data/retail_data_clean.csv')
retail_df_clean = prepare_retail_data(retail_df_clean)

# Chargement des DataSet pollués
pollution_percentage_levels = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
retail_df_noisy_dict = read_csv_files_to_dict('../../Data/Clustering/retail Data', 'Unicity', 'retail', pollution_percentage_levels)

for key in retail_df_noisy_dict.keys():
    if retail_df_noisy_dict[key] is not None:
        retail_df_noisy_dict[key] = prepare_retail_data(retail_df_noisy_dict[key])

metrics = compute_metrics(retail_df_clean, retail_df_noisy_dict)

update_json_results(output_path, model_name, metrics, pollution_percentage_levels)

Réduction de dimensionnalité sur les données propres
Clustering sur les données propres
Réduction de dimensionnalité sur les données bruitées: 10%
Clustering sur les données bruitées: 10%
Calcul du Adjusted Rand Index (ARI) sur les données bruitées
Calcul du Stability Index (SI) sur les données bruitées
Calcul de la taille absolue de l'overlap des clusters
Calcul de la variance de la taille des clusters
Réduction de dimensionnalité sur les données bruitées: 20%
Clustering sur les données bruitées: 20%
Calcul du Adjusted Rand Index (ARI) sur les données bruitées
Calcul du Stability Index (SI) sur les données bruitées
Calcul de la taille absolue de l'overlap des clusters
Calcul de la variance de la taille des clusters
Réduction de dimensionnalité sur les données bruitées: 30%
Clustering sur les données bruitées: 30%
Calcul du Adjusted Rand Index (ARI) sur les données bruitées
Calcul du Stability Index (SI) sur les données bruitées
Calcul de la taille absolue de l'overlap des clusters
Cal

## Test

In [2]:
df = pd.read_csv('../../Data/Clustering/retail Data/retail_data_clean.csv')
df

,City,State,Zipcode,Country,Age,Gender,Income,Customer_Segment,Total_Purchases,Amount,...,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products,Day,Month,Year
0,Dortmund,Berlin,77985.0,Germany,21.0,Male,Low,Regular,3.0,108.028757,...,Shorts,Excellent,Same-Day,Debit Card,Shipped,5.0,Cycling shorts,18,9,2023
1,Nottingham,England,99071.0,UK,19.0,Female,Low,Premium,2.0,403.353907,...,Tablet,Excellent,Standard,Credit Card,Processing,4.0,Lenovo Tab,31,12,2023
2,Geelong,New South Wales,75929.0,Australia,48.0,Male,Low,Regular,3.0,354.477600,...,Children's,Average,Same-Day,Credit Card,Processing,2.0,Sports equipment,26,4,2023
3,Edmonton,Ontario,88420.0,Canada,56.0,Male,High,Premium,7.0,352.407717,...,Tools,Excellent,Standard,PayPal,Processing,4.0,Utility knife,5,8,2023
4,Bristol,England,48704.0,UK,22.0,Male,Low,Premium,2.0,124.276524,...,Chocolate,Bad,Standard,Cash,Shipped,1.0,Chocolate cookies,1,10,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293906,Townsville,New South Wales,4567.0,Australia,31.0,Male,Medium,Regular,5.0,194.792597,...,Fiction,Bad,Same-Day,Cash,Processing,1.0,Historical fiction,20,1,2024
293907,Hanover,Berlin,16852.0,Germany,35.0,Female,Low,New,1.0,285.137301,...,Laptop,Excellent,Same-Day,Cash,Processing,5.0,LG Gram,28,12,2023
293908,Brighton,England,88038.0,UK,41.0,Male,Low,Premium,3.0,60.701761,...,Jacket,Average,Express,Cash,Shipped,2.0,Parka,27,2,2024
293909,Halifax,Ontario,67608.0,Canada,41.0,Male,Medium,New,1.0,120.834784,...,Furniture,Good,Standard,Cash,Shipped,4.0,TV stand,9,3,2023


In [3]:
# Séparation des caractéristiques numériques et catégorielles
numeric_features = df.select_dtypes(include='number')
categorical_features = df.select_dtypes(include='object')

# Transformation des données numériques
scaler = StandardScaler()
numeric_data_scaled = scaler.fit_transform(numeric_features)

# Transformation des données catégorielles en utilisant pd.get_dummies
categorical_data_encoded = pd.get_dummies(categorical_features, drop_first=True)

# Combinaison des données numériques et catégorielles transformées
df_preprocessed = pd.concat([pd.DataFrame(numeric_data_scaled, columns=numeric_features.columns).reset_index(drop=True),
                             categorical_data_encoded.reset_index(drop=True)], axis=1)

# Affichage des premières lignes du DataFrame prétraité
df_preprocessed

,Zipcode,Age,Total_Purchases,Amount,Total_Amount,Ratings,Day,Month,Year,City_Albuquerque,...,products_White chocolate,products_Wide-leg jeans,products_Windbreaker,products_Window AC,products_Wireless headphones,products_Wrap dress,products_Wrench,products_Xiaomi Mi,products_iPad,products_iPhone
0,0.955828,-0.963246,-0.822701,-1.040570,-0.924446,1.391396,0.254179,0.721840,-0.444729,False,...,False,False,False,False,False,False,False,False,False,False
1,1.683520,-1.096422,-1.171323,1.048181,-0.496928,0.634256,1.728691,1.590416,-0.444729,False,...,False,False,False,False,False,False,False,False,False,False
2,0.884874,0.834629,-0.822701,0.702493,-0.269515,-0.880025,1.161571,-0.725787,-0.444729,False,...,False,False,False,False,False,False,False,False,False,False
3,1.315947,1.367333,0.571788,0.687853,0.973668,0.634256,-1.220333,0.432314,-0.444729,False,...,False,False,False,False,False,False,False,False,False,False
4,-0.054678,-0.896658,-1.171323,-0.925654,-0.991355,-1.637165,-1.674029,1.011365,2.248561,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293906,-1.577875,-0.297366,-0.125457,-0.426914,-0.348770,-1.637165,0.481027,-1.594363,2.248561,False,...,False,False,False,False,False,False,False,False,False,False
293907,-1.153911,-0.031014,-1.519945,0.212068,-0.958948,1.391396,1.388419,1.590416,-0.444729,False,...,False,False,False,False,False,False,False,False,False,False
293908,1.302764,0.368513,-0.822701,-1.375301,-1.050216,-0.880025,1.274995,-1.304838,2.248561,False,...,False,False,False,False,False,False,False,False,False,False
293909,0.597711,0.368513,-1.519945,-0.949997,-1.104491,0.634256,-0.766637,-1.015312,-0.444729,False,...,False,False,False,False,False,False,False,False,False,False


In [4]:
# Réduction de la dimensionalité avec UMAP
umap = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, n_jobs=-1)  # Utilisation de tous les cœurs disponibles
embedding = umap.fit_transform(df_preprocessed)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [5]:
# Application du clustering HDBSCAN
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=10, min_samples=5, metric='euclidean')
cluster_labels = hdbscan_model.fit_predict(embedding)

In [6]:
# Nombre de clusters trouvés (en excluant le bruit)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
print(f"Nombre de clusters trouvés: {num_clusters}")
print(f"Distribution des clusters: {np.unique(cluster_labels, return_counts=True)}")

Nombre de clusters trouvés: 258
Distribution des clusters: (array([ -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
        12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,
        25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
        38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
        51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
        64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
        77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
       103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
       116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
       142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
       155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
    

In [7]:
# 3. Évaluation du Clustering

# AMI - Adjusted Mutual Information
# Vous devez avoir les étiquettes vraies (y_true) pour cette évaluation
# AMI est généralement utilisé lorsque vous avez des étiquettes de référence
# ami_score = adjusted_mutual_info_score(y_true, cluster_labels)

# Silhouette Score
silhouette_avg = silhouette_score(embedding, cluster_labels)

# Variance des tailles des clusters
cluster_sizes = np.array([count for label, count in Counter(cluster_labels).items() if label != -1])  # Ignorer les outliers
variance_cluster_size = np.var(cluster_sizes)

# Calcul de l'Overlap (en utilisant une approximation via la distance intra-cluster)
def calculate_overlap(labels, data):
    cluster_centers = []
    for label in np.unique(labels):
        if label != -1:  # Ignorer les outliers
            cluster_data = data[labels == label]
            center = np.mean(cluster_data, axis=0)
            cluster_centers.append(center)

    # Distance entre chaque cluster center
    distances = cdist(cluster_centers, cluster_centers, 'euclidean')
    np.fill_diagonal(distances, np.inf)
    overlap = np.sum(distances < np.mean(distances))

    return overlap

overlap_score = calculate_overlap(cluster_labels, embedding)

# Affichage des résultats
print(f'Silhouette Score: {silhouette_avg}')
print(f'Variance des tailles de clusters: {variance_cluster_size}')
print(f'Score Overlap: {overlap_score}')
# print(f'AMI: {ami_score}')  # Activer ceci si vous avez y_true

Silhouette Score: 0.39225906133651733
Variance des tailles de clusters: 11277709.093143443
Score Overlap: 66306


In [ ]:
import numpy as np
import pandas as pd
import umap
import hdbscan
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_mutual_info_score, silhouette_score, adjusted_rand_score, \
    normalized_mutual_info_score
from scipy.spatial.distance import cdist
from collections import Counter

def prepare_retail_data(df):
    # Séparation des caractéristiques numériques et catégorielles
    numeric_features = df.select_dtypes(include='number')
    categorical_features = df.select_dtypes(include='object')

    # Transformation des données numériques
    scaler = StandardScaler()
    numeric_data_scaled = scaler.fit_transform(numeric_features)

    # Transformation des données catégorielles en utilisant pd.get_dummies
    categorical_data_encoded = pd.get_dummies(categorical_features, drop_first=True)

    # Combinaison des données numériques et catégorielles transformées
    df_preprocessed = pd.concat([pd.DataFrame(numeric_data_scaled, columns=numeric_features.columns).reset_index(drop=True),
                                 categorical_data_encoded.reset_index(drop=True)], axis=1)

    return df_preprocessed

def compute_metrics(df_clean, df_noisy_dict, clustering_algo=hdbscan.HDBSCAN(min_cluster_size=10, min_samples=5)):
    metrics_results = {
        "Stability_Index": [],
        "ARI": [],
        "Silhouette_Score_Clean": None,
        "Silhouette_Score_Noisy": []
    }

    # 1. Réduction de dimensionnalité sur les données propres
    print("Réduction de dimensionnalité")
    umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2)
    embedding_clean = umap_model.fit_transform(df_clean)
    # 2. Clustering sur les données propres
    print("Clustering")
    cluster_labels_clean = clustering_algo.fit_predict(embedding_clean)

    # Calcul du Silhouette Score sur les données propres
    print("Calcul du Silhouette Score")
    silhouette_clean = silhouette_score(embedding_clean, cluster_labels_clean)
    metrics_results["Silhouette_Score_Clean"] = silhouette_clean

    for key in df_noisy_dict.keys():
        print(key)
        df_noisy = df_noisy_dict[key]

        if df_noisy is not None:
            # 3. Réduction de dimensionnalité sur les données bruitées
            embedding_noisy = umap_model.transform(df_noisy)

            # 4. Clustering sur les données bruitées
            cluster_labels_noisy = clustering_algo.fit_predict(embedding_noisy)

            # 5. Calcul des métriques

            # Adjusted Rand Index (ARI)
            ari = adjusted_rand_score(cluster_labels_clean, cluster_labels_noisy)
            metrics_results["ARI"].append(ari)

            # Silhouette Score sur les données bruitées
            silhouette_noisy = silhouette_score(embedding_noisy, cluster_labels_noisy)
            metrics_results["Silhouette_Score_Noisy"].append(silhouette_noisy)

            # Stability Index (SI) - en utilisant l'AMI comme proxy
            stability_index = normalized_mutual_info_score(cluster_labels_clean, cluster_labels_noisy)
            metrics_results["Stability_Index"].append(stability_index)
        else:
            # Si le dataframe n'est pas disponible, on ajoute None pour ce niveau de bruit
            metrics_results["ARI"].append(None)
            metrics_results["Silhouette_Score_Noisy"].append(None)
            metrics_results["Stability_Index"].append(None)

    return metrics_results

# Exemple d'utilisation
# Chargement du DataSet clean
retail_df_clean = pd.read_csv('../../Data/Clustering/retail Data/retail_data_clean.csv')
retail_df_clean = prepare_retail_data(retail_df_clean)

# Chargement des DataSet pollués
missing_percentages = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
retail_df_noisy_dict = read_csv_files_to_dict('../../Data/Clustering/retail Data', 'Feature Accuracy', 'retail', missing_percentages)

for key in retail_df_noisy_dict.keys():
    if retail_df_noisy_dict[key] is not None:
        retail_df_noisy_dict[key] = prepare_retail_data(retail_df_noisy_dict[key])

metrics = compute_metrics(retail_df_clean, retail_df_noisy_dict)

In [5]:
# Affichage des résultats
metrics

{'Stability_Index': [0.6827248633820063,
  0.43575938626099814,
  0.1721664525072392,
  0.16053127765393882,
  0.14671187464421312,
  0.13618995316396276,
  0.125958444437765,
  0.12020514270552757],
 'ARI': [0.43532076566190603,
  0.12633221200716088,
  0.031490041728447084,
  0.03005326058548987,
  0.027562095145463674,
  0.025858072089500632,
  0.023978171084148653,
  0.042972166286943655],
 'Silhouette_Score_Clean': 0.4796416,
 'Silhouette_Score_Noisy': [-0.027544223,
  -0.07346046,
  0.37912834,
  0.55803615,
  0.5646407,
  0.40605074,
  0.5764017,
  -0.07651526]}